In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from db_conn import engine, session_scope, ping_db, Base
from models import *

from eda_and_preprocessing.preprocessing_class import Preprocessing

In [2]:
ping_db(engine)

True

In [3]:
data = Preprocessing()

In [4]:
# create only the FactFunding table
Base.metadata.create_all(engine)

In [6]:
# data.locations_df --> DimensionLocation
with session_scope() as session:
    # Step 1: Insert or Fetch Continent Records
    continents = {}
    for continent_name in data.locations_df['Continent'].unique():
        continent = session.query(DimensionContinent).filter_by(continent_name=continent_name).first()
        if not continent:
            continent = DimensionContinent(continent_name=continent_name)
            session.add(continent)
            session.flush()  # Generate `continent_id` without committing
        continents[continent_name] = continent.continent_id

    # Step 2: Insert or Fetch Country Records
    countries = {}
    for _, row in data.locations_df[['Country', 'Continent']].drop_duplicates().iterrows():
        country_name = row['Country']
        continent_id = continents[row['Continent']]
        country = session.query(DimensionCountry).filter_by(country_name=country_name).first()
        if not country:
            country = DimensionCountry(country_name=country_name, continent_id=continent_id)
            session.add(country)
            session.flush()  # Generate `country_id` without committing
        countries[country_name] = country.country_id

    # Step 3: Populate DimensionLocation with Foreign Keys
    for _, row in data.locations_df.iterrows():
        country_id = countries[row['Country']]
        location = DimensionLocation(
            location_id=row['LocationID'],
            city=row['City'],
            country_id=country_id,
        )
        session.add(location)

    # Commit all changes
    session.commit()

In [7]:
# data.organizations_df --> DimensionOrganization
with session_scope() as session:
    for _, row in data.organizations_df.iterrows():
        location = DimensionOrganization(
        organization_id=row['OrganizationID'],
        organization_name=row['Organization Name'],
        location_id=row['LocationID'],
        organization_name_url=row['Organization Name URL'],
        organization_description=row['Organization Description'],
        organization_website=row['Organization Website']
        )
        session.add(location)
    session.commit()

In [8]:
# data.df --> FactFunding
with session_scope() as session:
    for _, row in data.df.iterrows():
        equity_only = row['Equity Only Funding'].lower() == 'true'
        funding = FactFunding(
            transaction_id=row['TransactionID'],
            organization_id=row['OrganizationID'],
            money_raised=row['Money Raised'],
            money_raised_usd=row['Money Raised (in USD)'],
            funding_type=row['Funding Type'],
            announced_date=row['Announced Date'],
            funding_stage=row['Funding Stage'],
            number_of_funding_rounds=row['Number of Funding Rounds'],
            total_funding_amount=row['Total Funding Amount'],
            total_funding_amount_usd=row['Total Funding Amount (in USD)'],
            equity_only=equity_only,
            transaction_name=row['Transaction Name'],
            transaction_name_url=row['Transaction Name URL'],
            money_raised_currency=row['Money Raised Currency'],
            pre_money_valuation=row['Pre-Money Valuation'],
            pre_money_valuation_currency=row['Pre-Money Valuation Currency'],
            pre_money_valuation_usd=row['Pre-Money Valuation (in USD)'],
            funding_status=row['Funding Status'],
            number_of_investors=row['Number of Investors']
        )
        session.add(funding)
    session.commit()

In [9]:
# data.investors_df --> DimensionInvestor
with session_scope() as session:
    for _, row in data.investors_df.iterrows():
        investor = DimensionInvestor(
            investor_id=row['InvestorID'],
            investor=row['Investor']
        )
        session.add(investor)
    session.commit()

In [10]:
# data.investor_mapping_df --> InvestorMapping
with session_scope() as session:
    for _, row in data.investor_mapping_df.iterrows():
        investor_mapping = InvestorMapping(
            transaction_id=int(row['TransactionID']),
            investor_id=int(row['InvestorID']),
            is_lead_investor=bool(row['IsLeadInvestor'])
        )
        session.add(investor_mapping)
    session.commit()

In [11]:
# data.industries_df --> DimensionIndustry
with session_scope() as session:
    for _, row in data.industries_df.iterrows():
        industry = DimensionIndustry(
            industry_id=row['IndustryID'],
            industry=row['Industry']
        )
        session.add(industry)
    session.commit()

In [12]:
# data.industry_mapping_df --> IndustryMapping
with session_scope() as session:
    for _, row in data.industry_mapping_df.iterrows():
        industry_mapping = IndustryMapping(
            organization_id=int(row['OrganizationID']),
            industry_id=int(row['IndustryID'])
        )
        session.add(industry_mapping)
    session.commit()

### Query Example:


In [13]:
# 1. Query all funding transactions
with session_scope() as session:
    all_fundings = session.query(FactFunding).all()
    for funding in all_fundings:
        print(f"Transaction ID: {funding.transaction_id}, Money Raised: {funding.money_raised}, Funding Type: {funding.funding_type}")

Transaction ID: 1, Money Raised: 2800000.0, Funding Type: Seed
Transaction ID: 2, Money Raised: 632000.0, Funding Type: Seed
Transaction ID: 3, Money Raised: 23500000.0, Funding Type: Seed
Transaction ID: 4, Money Raised: 21956027.0, Funding Type: Pre-Seed
Transaction ID: 5, Money Raised: 25000.0, Funding Type: Seed
Transaction ID: 6, Money Raised: 500000.0, Funding Type: Pre-Seed
Transaction ID: 7, Money Raised: 20000.0, Funding Type: Pre-Seed
Transaction ID: 8, Money Raised: 5487000.0, Funding Type: Seed
Transaction ID: 9, Money Raised: 217519.0, Funding Type: Seed
Transaction ID: 10, Money Raised: 2850000.0, Funding Type: Seed
Transaction ID: 11, Money Raised: 1325000.0, Funding Type: Pre-Seed
Transaction ID: 12, Money Raised: 5000000.0, Funding Type: Seed
Transaction ID: 13, Money Raised: 2000000.0, Funding Type: Seed
Transaction ID: 14, Money Raised: 500000.0, Funding Type: Seed
Transaction ID: 15, Money Raised: 100671.0, Funding Type: Pre-Seed
Transaction ID: 16, Money Raised: 25

In [17]:
# 2. Query all investors for a specific transaction
with session_scope() as session:
    transaction_id = 1 
    investors_for_transaction = session.query(DimensionInvestor).join(InvestorMapping).filter(InvestorMapping.transaction_id == transaction_id).all()
    for investor in investors_for_transaction:
        print(f"Investor ID: {investor.investor_id}, Investor Name: {investor.investor}")

Investor ID: 1, Investor Name: Charles Delingpole
Investor ID: 2, Investor Name: Donald Bringmann
Investor ID: 3, Investor Name: Erik Muttersbach
Investor ID: 4, Investor Name: Four Cities Capital
Investor ID: 5, Investor Name: Fredrik Thomassen
Investor ID: 6, Investor Name: Kevin Costa
Investor ID: 7, Investor Name: Louis Beryl
Investor ID: 8, Investor Name: Moonfire Ventures
Investor ID: 9, Investor Name: Pioneer Fund
Investor ID: 10, Investor Name: Sundeep Ahuja
Investor ID: 11, Investor Name: Y Combinator


In [18]:
# 3. Query funding rounds for organizations in a specific industry
with session_scope() as session:
    industry_name = 'FinTech'  # Example industry
    fundings_in_industry = session.query(FactFunding).join(DimensionOrganization).join(IndustryMapping).join(DimensionIndustry).filter(DimensionIndustry.industry == industry_name).all()
    for funding in fundings_in_industry:
        print(f"Organization ID: {funding.organization_id}, Money Raised: {funding.money_raised_usd}, Industry: {industry_name}")

Organization ID: 1, Money Raised: 2800000.0, Industry: FinTech
Organization ID: 13, Money Raised: 20000.0, Industry: FinTech
Organization ID: 20, Money Raised: 5000000.0, Industry: FinTech
Organization ID: 27, Money Raised: 3956926.0, Industry: FinTech
Organization ID: 29, Money Raised: 2000000.0, Industry: FinTech
Organization ID: 35, Money Raised: 5500000.0, Industry: FinTech
Organization ID: 55, Money Raised: 3233019.0, Industry: FinTech
Organization ID: 85, Money Raised: 3750000.0, Industry: FinTech
Organization ID: 100, Money Raised: 2922393.0, Industry: FinTech
Organization ID: 109, Money Raised: 1700000.0, Industry: FinTech
Organization ID: 122, Money Raised: 250000.0, Industry: FinTech
Organization ID: 133, Money Raised: 6400000.0, Industry: FinTech
Organization ID: 153, Money Raised: 16637293.0, Industry: FinTech
Organization ID: 173, Money Raised: 130000.0, Industry: FinTech
Organization ID: 196, Money Raised: 2039454.0, Industry: FinTech
Organization ID: 216, Money Raised: 3

In [19]:
# 4. Query lead investors in a specific transaction
with session_scope() as session:
    lead_investors = session.query(DimensionInvestor).join(InvestorMapping).filter(InvestorMapping.transaction_id == transaction_id, InvestorMapping.is_lead_investor == True).all()
    for lead_investor in lead_investors:
        print(f"Lead Investor ID: {lead_investor.investor_id}, Lead Investor Name: {lead_investor.investor}")

Lead Investor ID: 8, Lead Investor Name: Moonfire Ventures
Lead Investor ID: 11, Lead Investor Name: Y Combinator
